#1. Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

# first install yahoo finance
!pip install yfinance
!pip install yahoofinancials
import yfinance as yf
from yahoofinancials import YahooFinancials
# used to calculate technical indicators
!pip install stockstats
import stockstats

# 

#2. Download Financial Data

In [2]:
data = yf.download(
  tickers = "AXP AMGN AAPL BA CAT CSCO CVX GS HD HON IBM INTC JNJ KO JPM MCD MMM MRK MSFT NKE PG TRV UNH CRM VZ V WBA WMT DIS DOW", # Dow Jones 30 Tickers
  start="2008-01-01",
  end = "2022-11-30",
  interval="1d",
  ignore_tz=True,
  group_by='ticker',
  auto_adjust= True,
  repair=True,
  prepost=False,
  threads=True,
  proxy=None
)

[*********************100%***********************]  30 of 30 completed


In [3]:
data.head()

CRM                                            TRV             \
               Open     High      Low   Close    Volume       Open       High   
Date                                                                            
2008-01-02  15.7625  15.8875  14.9675  15.070   4980000  37.233770  37.261526   
2008-01-03  15.0000  15.4575  15.0000  15.245   3689200  36.331718  37.004785   
2008-01-04  15.0025  15.1525  14.5200  14.715   6438000  36.512133  36.817443   
2008-01-07  14.4800  14.5050  13.7150  14.440  13106800  35.936213  36.505197   
2008-01-08  14.4400  14.8225  13.6275  13.675   6574000  36.220704  36.519074   

                                           ...        MMM             \
                  Low      Close   Volume  ...       Open       High   
Date                                       ...                         
2008-01-02  36.179067  36.331722  2341400  ...  54.843365  55.181907   
2008-01-03  36.303962  36.817436  2098800  ...  53.918886  54.348572   
2008-01-04  35.617026  35.658657  3048900  ...  53.385068  53.814755   
2008-01-07  35.401923  36.109684  3130300  ...  53.450159  53.482713   
2008-01-08  34.846816  35.006409  4684600  ...  52.961882  53.430630   

                                                 HON                        \
                  Low      Close   Volume       Open       High        Low   
Date                                                                         
2008-01-02  53.534784  53.847279  4453700  41.529025  41.529025  40.313934   
2008-01-03  53.625921  53.840759  2724100  40.489453  40.820224  40.219433   
2008-01-04  53.092104  53.228821  3620300  40.212690  40.617716  39.233866   
2008-01-07  52.584278  52.805634  4223800  39.503893  39.706408  38.390062   
2008-01-08  52.148085  52.219700  5711200  39.362133  39.483643  38.477819   

                                
                Close   Volume  
Date                            
2008-01-02  40.435444  4365569  
2008-01-03  40.637962  3571018  
2008-01-04  39.341873  6237770  
2008-01-07  39.206871  5873483  
2008-01-08  38.538574  5551887  

[5 rows x 150 columns]

In [4]:
data.shape

(3755, 150)

There is OHLCV (Open, High, Low, Close, Volume) data for 30 stocks = 150 columns, and 3502 days (rows)

# 3. Preprocess Data

Important steps:

1. Check for missing data

2. Add chosen technical indicators to use as features

In [5]:
data_copy = data.copy()

Check for missing values

In [6]:
data_copy.isnull().sum()[data_copy.isnull().sum() != 0]

V    Open        53
     High        53
     Low         53
     Close       53
     Volume      53
DOW  Open      2822
     High      2822
     Low       2822
     Close     2822
     Volume    2822
dtype: int64

DOW and V are missing significant number of days -> drop them from the dataset, leaving us with 28 stocks

In [7]:
ticker_list = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 
               'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 
               'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'WBA', 'WMT', 
               'DIS'] # dropped V and DOW

In [8]:
from stockstats import wrap

def add_indicators_reshape_df(df, ticker_list, indicator_list):
  stock_df_list = []
  for i in ticker_list:
    temp = df[i]
    temp['tick'] = i

    # get and add indicators to the dataframe
    temp_w = wrap(temp)
    t = temp.merge(temp_w[indicator_list], how="left")
    t.set_index(temp.index, inplace=True)

    stock_df_list.append(t)

  new_df = pd.concat(stock_df_list, sort=False).sort_index()
  return new_df

In [9]:
data_w_ind = add_indicators_reshape_df(data_copy, ticker_list, indicator_list=['macd', 'rsi', 'cci'])

C:\Users\benja\AppData\Local\Temp\ipykernel_6344\1553338203.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['tick'] = i
C:\Users\benja\AppData\Local\Temp\ipykernel_6344\1553338203.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['tick'] = i
C:\Users\benja\AppData\Local\Temp\ipykernel_6344\1553338203.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

Check for missing values

In [10]:
data_w_ind.head()

,open,high,low,close,volume,tick,macd,macds,macdh,rs_14,rsi,cci
Date,,,,,,,,,,,,
2008-01-02,40.999957,41.180989,39.976729,40.173504,8053700.0,AXP,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,64.177842,64.375716,63.027229,63.481613,4303000.0,BA,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,27.419730,27.682730,26.885249,27.012506,9269900.0,DIS,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,47.810805,47.883286,46.156934,46.539101,6337800.0,CAT,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,19.173512,19.386551,18.612509,18.846853,64338900.0,CSCO,0.0,0.0,0.0,NaN,NaN,NaN


In [11]:
data_w_ind[data_w_ind.isna().any(axis=1)]

,open,high,low,close,volume,tick,macd,macds,macdh,rs_14,rsi,cci
Date,,,,,,,,,,,,
2008-01-02,40.999957,41.180989,39.976729,40.173504,8.053700e+06,AXP,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,64.177842,64.375716,63.027229,63.481613,4.303000e+06,BA,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,27.419730,27.682730,26.885249,27.012506,9.269900e+06,DIS,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,47.810805,47.883286,46.156934,46.539101,6.337800e+06,CAT,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,19.173512,19.386551,18.612509,18.846853,6.433890e+07,CSCO,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,52.506498,52.914436,51.807976,52.227089,9.058000e+06,CVX,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,171.201391,171.400647,164.665772,165.462799,9.573500e+06,GS,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,34.043747,34.201724,33.361579,33.677532,1.991860e+07,WMT,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,18.553060,18.648801,17.793979,17.855526,1.656070e+07,HD,0.0,0.0,0.0,NaN,NaN,NaN


NaN values for first two days in the dataset, therefore drop these days.

In [12]:
df = data_w_ind.copy()

df = df.drop([pd.to_datetime('2008-01-02'), pd.to_datetime('2008-01-03')])
df

,open,high,low,close,volume,tick,macd,macds,macdh,rs_14,rsi,cci
Date,,,,,,,,,,,,
2008-01-04,42.310987,42.748776,42.310987,42.388241,12714800.0,JNJ,-0.001429,-0.000491,-0.000938,0.206105,17.088460,-24.850656
2008-01-04,46.034176,46.862406,46.034176,46.239628,10049000.0,PG,-0.005774,-0.002366,-0.003407,0.000000,0.000000,-100.000000
2008-01-04,53.385068,53.814755,53.092104,53.228821,3620300.0,MMM,-0.019163,-0.007901,-0.011261,0.000000,0.000000,-100.000000
2008-01-04,36.512133,36.817443,35.617026,35.658657,3048900.0,TRV,-0.022048,-0.005463,-0.016585,0.389220,28.017165,-100.000000
2008-01-04,26.299861,26.876762,26.299861,26.410151,9550700.0,DIS,-0.018535,-0.008033,-0.010502,0.000000,0.000000,-100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-29,531.669983,532.969971,523.650024,528.000000,3751000.0,UNH,-0.417645,-0.561892,0.144247,0.991349,49.782786,15.513895
2022-11-29,152.720001,154.020004,151.259995,151.679993,6398100.0,CRM,-0.493688,-0.601808,0.108121,0.964399,49.093839,-0.847816
2022-11-29,38.020000,38.369999,37.950001,38.340000,14008800.0,VZ,0.272868,0.201219,0.071649,1.096433,52.299936,-34.559979


Make date a column and make a column called day that starts at 1 and increments upwards.

In [46]:
df_c = df.copy()
df_c['Date'] = df.index

df_c['day'] = 0
day = 0
for i in range(1, df_c.shape[0]):
    if df_c.iloc[i-1,-2] == df_c.iloc[i,-2]:
        df_c.iloc[i,-1] = df_c.iloc[i-1,-1]
    else:
        day += 1
        df_c.iloc[i,-1] = day

In [47]:
df_c

,open,high,low,close,volume,tick,macd,macds,macdh,rs_14,rsi,cci,Date,day
Date,,,,,,,,,,,,,,
2008-01-04,42.310987,42.748776,42.310987,42.388241,12714800.0,JNJ,-0.001429,-0.000491,-0.000938,0.206105,17.088460,-24.850656,2008-01-04,0
2008-01-04,46.034176,46.862406,46.034176,46.239628,10049000.0,PG,-0.005774,-0.002366,-0.003407,0.000000,0.000000,-100.000000,2008-01-04,0
2008-01-04,53.385068,53.814755,53.092104,53.228821,3620300.0,MMM,-0.019163,-0.007901,-0.011261,0.000000,0.000000,-100.000000,2008-01-04,0
2008-01-04,36.512133,36.817443,35.617026,35.658657,3048900.0,TRV,-0.022048,-0.005463,-0.016585,0.389220,28.017165,-100.000000,2008-01-04,0
2008-01-04,26.299861,26.876762,26.299861,26.410151,9550700.0,DIS,-0.018535,-0.008033,-0.010502,0.000000,0.000000,-100.000000,2008-01-04,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-29,531.669983,532.969971,523.650024,528.000000,3751000.0,UNH,-0.417645,-0.561892,0.144247,0.991349,49.782786,15.513895,2022-11-29,3752
2022-11-29,152.720001,154.020004,151.259995,151.679993,6398100.0,CRM,-0.493688,-0.601808,0.108121,0.964399,49.093839,-0.847816,2022-11-29,3752
2022-11-29,38.020000,38.369999,37.950001,38.340000,14008800.0,VZ,0.272868,0.201219,0.071649,1.096433,52.299936,-34.559979,2022-11-29,3752


In [48]:
df_c = df_c.reset_index(drop=True)

In [49]:
df_c = df_c.set_index(df_c['day'])

In [50]:
df_c

,open,high,low,close,volume,tick,macd,macds,macdh,rs_14,rsi,cci,Date,day
day,,,,,,,,,,,,,,
0,42.310987,42.748776,42.310987,42.388241,12714800.0,JNJ,-0.001429,-0.000491,-0.000938,0.206105,17.088460,-24.850656,2008-01-04,0
0,46.034176,46.862406,46.034176,46.239628,10049000.0,PG,-0.005774,-0.002366,-0.003407,0.000000,0.000000,-100.000000,2008-01-04,0
0,53.385068,53.814755,53.092104,53.228821,3620300.0,MMM,-0.019163,-0.007901,-0.011261,0.000000,0.000000,-100.000000,2008-01-04,0
0,36.512133,36.817443,35.617026,35.658657,3048900.0,TRV,-0.022048,-0.005463,-0.016585,0.389220,28.017165,-100.000000,2008-01-04,0
0,26.299861,26.876762,26.299861,26.410151,9550700.0,DIS,-0.018535,-0.008033,-0.010502,0.000000,0.000000,-100.000000,2008-01-04,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3752,531.669983,532.969971,523.650024,528.000000,3751000.0,UNH,-0.417645,-0.561892,0.144247,0.991349,49.782786,15.513895,2022-11-29,3752
3752,152.720001,154.020004,151.259995,151.679993,6398100.0,CRM,-0.493688,-0.601808,0.108121,0.964399,49.093839,-0.847816,2022-11-29,3752
3752,38.020000,38.369999,37.950001,38.340000,14008800.0,VZ,0.272868,0.201219,0.071649,1.096433,52.299936,-34.559979,2022-11-29,3752


Set stock_df to be the final dataframe

In [ ]:
stock_df = df_c.copy()

# 4. Design the RL Environment

Use OpenAI's Gym framework to build the environment.

In [ ]:
day = stock_df.loc[0,:]

indicator_list=['macd', 'rsi', 'cci']

state = (
    [10000]
    + day.close.values.tolist()
    + [0]*28
    + sum((day[tech].values.tolist() for tech in indicator_list), [])
)

state = (
    [10000]
    + day.close.tolist()
    + [0]*28
    + sum(([day[tech]] for tech in indicator_list), [])
)

state[28]

12.804293632507324

In [ ]:
import gym
from gym import spaces

!pip install stable_baselines3
from stable_baselines3 import DQN

class StockTradingEnv(gym.Env):
    def __init__(self,
               df,
               num_stocks,
               tmax,
               initial_bal,
               state_space_dim,
               action_space_dim,
               tech_indicators,
               fractional_shares=False,
               day=0,
               initial=True):
        self.df = df
        self.num_stocks = num_stocks
        self.tmax = tmax    # set parameter that defines maximum number of shares that can be bought and sold
        self.initial_bal = initial_bal
        self.state_space_dim = state_space_dim
        self.action_space_dim = action_space_dim
        self.tech_indicators = tech_indicators
        self.fractional_shares = fractional_shares  # indicates if shares bought/sold can be fractional
        self.day = day

        self.data = self.df.loc[self.day, :]

        self.state = self._create_state()
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.action_space_dim,))

        self.actions_history = []    # list to hold all actions taken in episode
        self.asset_history = []  # list to keep track of total asset value at each step
        self.date_history = []
        self.reward_history = []
        self.state_history = []

        # values for reporting at end
        self.trades = 0 # count total trades

    def _create_state(self):
        """
        Function to create the beginning state for the system

        The state is a 3 column vector:
            Remaining balance
            Current price of each stock (float * num_stocks)
            Shares held of each stock (float * num_stocks)
            Technical indicators (len(tech_indicators) * num_stocks)
        """
        if self.initial:
            state = (
                [self.initial_bal]
                + [self.data.close.values.tolist()]
                + [0]*self.num_stocks
                + sum((self.data[tech].values.tolist() for tech in self.tech_indicators), [])
            )

        return state

    def _update_state(self):
        state = (
            [self.state[0]]
            + [self.data.close.values.tolist()]
            + list(self.state[(self.num_stocks + 1) : (self.stateself.num_stocks * 2 + 1)])
            + sum((self.data[tech].values.tolist() for tech in self.tech_indicators), [])
        )

        return state

    def _sell_stock(self, index, action):
        """
            Method to sell a stock

            Check if the we currently have any of the stock.
        """
        if self.state[self.num_stocks + index + 1] > 0:
            # sell only if we have any of the stock
            num_shares_sell = min(
                abs(action), self.state[self.num_stocks + index + 1]
            )   # can sell at most the number of shares we currently own
            sell_value = (self.state[index + 1] * num_shares_sell)
            self.state[0] += sell_value

            self.state[index + self.num_stocks + 1] -= num_shares_sell
            self.trades += 1
        else:
            num_shares_sell = 0

        return num_shares_sell()

    def _buy_stocks(self, index, action):
        if self.state[index + 1] > 0:
            # only buy if there is an actual buy price
            amount_avail = self.state[0] // self.state[index + 1]
        
            num_shares_buy = min(amount_avail, action)  # can buy max of amount_avail
            buy_value = (self.state[index + 1] * num_shares_buy)
            self.state[0] -= buy_value

            self.state[index + self.num_stocks + 1] += num_shares_buy
        else:
            num_shares_buy = 0

        return num_shares_buy

    def _asset_total(self):
        total_asset_val = self.state[0] + sum(
                np.array(self.state[1 : (self.num_stocks + 1)]) *
                np.array(self.state[(1 + self.num_stocks) : (self.num_stocks *2 + 1)])
            )
        return total_asset_val

    def step(self, actions):
        # check for terminal state
        self.terminal = self.day >= len(self.df.index.unique()) - 1

        if self.terminal:
            # calculate the total value of assets at end of episode
            final_total_assets = self._asset_total()
            total_reward = final_total_assets - self.initial_bal    # total gain

            return self.state, self.reward, self.terminal
        
        else:
            actions = actions * self.tmax   # scale {0,1} action
            if not self.fractional_shares:
                actions = actions.astype(int)   # can't buy fractional shares
            start_asset_total = self._asset_total()

            # sort actions small to big
            argsort_actions = np.argsort(actions)
            sell_inds = argsort_actions[: np.where(actions < 0)[0].shape[0]]
            buy_inds = argsort_actions[::-1][: np.where(actions > 0)[0].shape[0]]   # reverse list so biggest positive actions are first

            for ind in sell_inds:
                actions[ind] = self._sell_stock(ind, actions[ind]) * (-1)
            for ind in buy_inds:
                actions[ind] = self._buy_stocks(ind, actions[ind])

            self.actions_history.append(actions) # record actions

            # increment to next state
            self.day += 1
            self.data = self.df.loc[self.day, :]
            self._update_state()

            end_asset_total = self._asset_total()

            self.asset_history.append(end_asset_total)
            self.date_history.append(self._get_date())
            self.reward = self.end_asset_total - self.start_asset_total
            self.reward_history.append(self.reward)
            self.state_history.append(self.state)
            
        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.state = self._create_state()
        self.asset_history = [self.initial_bal]
        self.day = 0
        self.data = self.df.loc[self.day, :]
        

        







